In [1]:
import uvicorn
from fastapi import FastAPI
import json
from typing import Union, List


from db_connectors import PostgresConnection, pg_pool
from schemas import User_Query
from query_search import fields_update, search_query


In [2]:
# import uvicorn
# from fastapi import FastAPI
import json
from typing import Union, List


from db_connectors import PostgresConnection, pg_pool
from schemas import User_Query
from query_search import fields_update, search_query


# app = FastAPI()

pg = PostgresConnection(pg_pool)

In [3]:
# @app.get('/')
def home():
    return {'key':'Hello world2 '}


# @app.get('/filters')
def filters():
    query_system = """
    SELECT DISTINCT
        it_code,
        name_system,
        status,
        owner_company,
        international_name,
        identifier,
        sg_global_id,
        alternative_name,
        producer,
        domain_owner_it_system,
        it_system_owner,
        responsible_employee_system_owner,
        development_department,
        responsible_for_development,
        application_manager,
        information_security_officer,
        "date",
        development_start_date,
        commissioning_date,
        start_date_shutdown,
        decommissioning_date,
        type_of_use,
        development_type,
        hosting_type,
        access_category,
        sensitivity,
        sensitivity_assessment_date,
        pci_dss_processing,
        green_book_compliance,
        internet_access,
        fraud_sensitive,
        integirty,
        confidentiality,
        availability,
        sensivity_category,
        vital,
        rto_target,
        rpo_target,
        note,
        last_modified_date
    FROM dq_sbx.registry_it_system
    WHERE  it_code in (SELECT distinct it_code FROM dq_sbx.descr_tab)
    ORDER BY name_system
    """
    query_owner = """
    SELECT contact_person
    FROM dq_sbx.descr_col
    WHERE contact_person IS NOT NULL
    GROUP BY contact_person
    ORDER BY contact_person
    """
    query_domain = """
    SELECT domain_data_owner
    FROM dq_sbx.descr_col
    WHERE domain_data_owner IS NOT NULL
    GROUP BY domain_data_owner
    ORDER BY domain_data_owner
    """
    system = pg.getData(query_system).to_dict('list')
    owner = pg.getData(query_owner).to_dict('list')
    domain = pg.getData(query_domain).to_dict('list')
    return system, owner, domain


# @app.post('/query')
def create_item(item: User_Query):
    query = User_Query(**item)
    query = fields_update(query)
    return pg.getData(search_query(query)).to_dict('list') 

In [64]:
# @app.post('/query')
def create_item(item: User_Query):
    query = User_Query(**item)
    query = fields_update(query)
    return pg.getData(search_query(query)).columns.tolist(), pg.getData(search_query(query)).values.tolist()

# search_query(query) #

search = {
    "query":"dq_sbx.profiling_columns",
    "fields_tab":{"table", "tab_comment"},
    "fields_col":{"col_name", "col_comment"}
    }


search ={
    "query":"        descr'    ",
    "odpp":True,
    "fields_tab":["table", "tab_comment"],
    "fields_col":["col_name", "col_comment","entity_name"],
    "filters_tab":[
        {
            "field":"it_code",
            "value": ["ci467113","ci22222"]
            }
        ]
    , 
    "filters_col":[
        {
            "field":"contact_person",
            "value": ["розумеенко марина сергеевна","test" ]
        }
        , 
        {
            "field":"domain_data_owner",
            "value": ["data","test" ]
        } 
        ]     
        ,
    "limit": 15
}


In [67]:
User_Query(**search)

User_Query(query='descr', odpp=True, fields_tab={'table', 'tab_comment'}, fields_col={'entity_name', 'col_name', 'col_comment'}, filters_tab=[Filters_Dict(field='it_code', value=['ci467113', 'ci22222'])], filters_col=[Filters_Dict(field='contact_person', value=['розумеенко марина сергеевна', 'test']), Filters_Dict(field='domain_data_owner', value=['data', 'test'])], limit=15)

In [59]:
print(create_item(search))


 
        
        WITH filters_col AS (
        SELECT table_code FROM dq_sbx.descr_col WHERE 1=1 
         AND contact_person IN ('розумеенко марина сергеевна', 'test') AND domain_data_owner IN ('data', 'test')
        GROUP BY table_code
        )
        
            SELECT 
            it_code, src_table, max(tab_comment) as tab_comment, table_code, 
            CASE 
                WHEN schema_name||'.'||table_name='descr' THEN schema_name||'.'||table_name
                WHEN src_schema||'.'||src_table='descr' THEN src_schema||'.'||src_table
                ELSE NULL
            END as search_val
            FROM dq_sbx.descr_tab 
            WHERE  1=1  
                AND (schema_name||'.'||table_name='descr' OR src_schema||'.'||src_table= 'descr')
                 AND it_code IN ('ci467113', 'ci22222') 
                 AND table_name IS NOT NULL  
                 AND table_code in (SELECT table_code FROM filters_col ) 
            GROUP BY it_code, src_table, table_code, 

In [60]:
s = """{"detail":[{"loc":["body",47],"msg":"Expecting value: line 3 column 12 (char 47)","type":"value_error.jsondecode","ctx":{"msg":"Expecting value","doc":"{\n    \"query\":\"        descr'    \",\n    \"odpp\":True,\n    \"fields_tab\":[\"table\", \"tab_comment\"],\n    \"fields_col\":[\"col_name\", \"col_comment\",\"entity_name\"],\n    \"filters_tab\":[\n        {\n            \"field\":\"it_code\",\n            \"value\": [\"ci467113\",\"ci22222\"]\n            }\n        ]\n    , \n    \"filters_col\":[\n        {\n            \"field\":\"contact_person\",\n            \"value\": [\"розумеенко марина сергеевна\",\"test\" ]\n        }\n        , \n        {\n            \"field\":\"domain_data_owner\",\n            \"value\": [\"data\",\"test\" ]\n        } \n        ]     \n        ,\n    \"limit\": 15\n}","pos":47,"lineno":3,"colno":12}}]}"""

In [61]:
print(s)

{"detail":[{"loc":["body",47],"msg":"Expecting value: line 3 column 12 (char 47)","type":"value_error.jsondecode","ctx":{"msg":"Expecting value","doc":"{
    "query":"        descr'    ",
    "odpp":True,
    "fields_tab":["table", "tab_comment"],
    "fields_col":["col_name", "col_comment","entity_name"],
    "filters_tab":[
        {
            "field":"it_code",
            "value": ["ci467113","ci22222"]
            }
        ]
    , 
    "filters_col":[
        {
            "field":"contact_person",
            "value": ["розумеенко марина сергеевна","test" ]
        }
        , 
        {
            "field":"domain_data_owner",
            "value": ["data","test" ]
        } 
        ]     
        ,
    "limit": 15
}","pos":47,"lineno":3,"colno":12}}]}
